# Use framevork BERT for recognition name, subname, pytromics 

In [1]:
import pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [2]:
data_sources = ['/home/al/Projects_My/NLP-russian-language/datasets/DataSet_servis/data_kazan/yandex_executor_viwes_kazan.csv',
               '/home/al/Projects_My/NLP-russian-language/datasets/DataSet_servis/data_chelyabinsk/yandex_executor_viwes_chelyabinsk.csv']

In [3]:
def preprocess_text(sen):

    # Remove punctuations and numbers
    sentence = re.sub('[^а-яёА-ЯЁ]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[а-яёА-ЯЁ]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence.lower()

In [4]:
data = pd.read_csv(data_sources[0], sep=';', header=None, on_bad_lines='skip')
print(data.head())
print(data.info())

                                          0  1           2  \
0                      RuslanKutliev-269762  5  13.11.2019   
1                      RuslanKutliev-269762  5  12.11.2019   
2                      RuslanKutliev-269762  5  12.11.2019   
3  VashEhlektrikEhlektromontazhKazan-100833  5  27.01.2020   
4  VashEhlektrikEhlektromontazhKazan-100833  5  23.01.2020   

                                                   3  
0  Ничего не могу сказать о нем плохого..Он круто...  
1  Обратился к Руслану,остался доволен. Оперативн...  
2  Руслан приехал вовремя, Собрал стол, повесил к...  
3                       Отличные ребята! Рекомендую.  
4  Решили с мужем делать ремонт в квартире. В пер...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23794 entries, 0 to 23793
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       23794 non-null  object
 1   1       23794 non-null  int64 
 2   2       23539 non-null  object
 3   3   

In [5]:
data_text = data[[1,3]]
print(data_text.head())

   1                                                  3
0  5  Ничего не могу сказать о нем плохого..Он круто...
1  5  Обратился к Руслану,остался доволен. Оперативн...
2  5  Руслан приехал вовремя, Собрал стол, повесил к...
3  5                       Отличные ребята! Рекомендую.
4  5  Решили с мужем делать ремонт в квартире. В пер...


In [6]:
data_text[1].unique()

array([5, 1, 4, 0, 2, 3])

In [7]:
print(data_text[data_text[1] ==0].count())
print(data_text[data_text[1] ==1].count())
print(data_text[data_text[1] ==2].count())
print(data_text[data_text[1] ==3].count())
print(data_text[data_text[1] ==4].count())
print(data_text[data_text[1] ==5].count())

1    76
3    76
dtype: int64
1    1015
3    1015
dtype: int64
1    104
3    104
dtype: int64
1    173
3    173
dtype: int64
1    457
3    457
dtype: int64
1    21969
3    21969
dtype: int64


In [8]:
full_text =''
for dd in data_text[3]:
    full_text += dd.replace('\n', ' ')
print(len(full_text))

5363826


In [10]:
reviews = []
sentences = list(data_text[3])
print(sentences[10])
for sen in sentences:
    reviews.append(preprocess_text(sen))
print(len(sentences))
print(reviews[10])

Электрик Ренат от них выезжал к моим родителям за город в Зеленодольский район в село Ключи. Там у моих родителей дом. Были постоянные проблемы с электрикой. Она была просто в ужасном состояний. Ренат все починил на совесть. Прошло уж года четыре наверное. Все нормально работает до сейх пор. Не скрываю советую его своим знакомым.
23794
электрик ренат от них выезжал моим родителям за город зеленодольский район село ключи там моих родителей дом были постоянные проблемы электрикой она была просто ужасном состояний ренат все починил на совесть прошло уж года четыре наверное все нормально работает до сейх пор не скрываю советую его своим знакомым 


In [11]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

Using /tmp/tfhub_modules to cache modules.
2022-01-15 22:19:32.670653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-15 22:19:32.696122: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-01-15 22:19:32.696137: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-01-15 22:19:32.696493: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with

In [12]:
tokenizer.tokenize(reviews[10])

['э',
 '##л',
 '##е',
 '##к',
 '##т',
 '##р',
 '##и',
 '##к',
 'р',
 '##е',
 '##на',
 '##т',
 'о',
 '##т',
 'н',
 '##и',
 '##х',
 'в',
 '##ы',
 '##е',
 '##з',
 '##ж',
 '##а',
 '##л',
 'м',
 '##о',
 '##и',
 '##м',
 'р',
 '##о',
 '##д',
 '##и',
 '##т',
 '##е',
 '##л',
 '##я',
 '##м',
 'з',
 '##а',
 'г',
 '##о',
 '##р',
 '##о',
 '##д',
 'з',
 '##е',
 '##л',
 '##е',
 '##н',
 '##о',
 '##д',
 '##о',
 '##л',
 '##ь',
 '##с',
 '##к',
 '##ии',
 'р',
 '##а',
 '##и',
 '##о',
 '##н',
 'с',
 '##е',
 '##л',
 '##о',
 'к',
 '##л',
 '##ю',
 '##ч',
 '##и',
 'т',
 '##а',
 '##м',
 'м',
 '##о',
 '##и',
 '##х',
 'р',
 '##о',
 '##д',
 '##и',
 '##т',
 '##е',
 '##л',
 '##е',
 '##и',
 'д',
 '##о',
 '##м',
 'б',
 '##ы',
 '##л',
 '##и',
 'п',
 '##о',
 '##с',
 '##т',
 '##о',
 '##я',
 '##н',
 '##н',
 '##ы',
 '##е',
 'п',
 '##р',
 '##о',
 '##б',
 '##л',
 '##е',
 '##м',
 '##ы',
 'э',
 '##л',
 '##е',
 '##к',
 '##т',
 '##р',
 '##и',
 '##к',
 '##о',
 '##и',
 'о',
 '##на',
 'б',
 '##ы',
 '##л',
 '##а',
 'п',
 '##р',
 '##о

In [13]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(reviews[10]))

[1208,
 29436,
 15290,
 23925,
 22919,
 16856,
 10325,
 23925,
 1195,
 15290,
 19865,
 22919,
 1193,
 22919,
 1192,
 10325,
 29750,
 1182,
 29113,
 15290,
 29744,
 29743,
 10260,
 29436,
 1191,
 14150,
 10325,
 29745,
 1195,
 14150,
 29742,
 10325,
 22919,
 15290,
 29436,
 17432,
 29745,
 1187,
 10260,
 1183,
 14150,
 16856,
 14150,
 29742,
 1187,
 15290,
 29436,
 15290,
 18947,
 14150,
 29742,
 14150,
 29436,
 23742,
 29747,
 23925,
 15414,
 1195,
 10260,
 10325,
 14150,
 18947,
 1196,
 15290,
 29436,
 14150,
 1189,
 29436,
 29757,
 29752,
 10325,
 1197,
 10260,
 29745,
 1191,
 14150,
 10325,
 29750,
 1195,
 14150,
 29742,
 10325,
 22919,
 15290,
 29436,
 15290,
 10325,
 1184,
 14150,
 29745,
 1181,
 29113,
 29436,
 10325,
 1194,
 14150,
 29747,
 22919,
 14150,
 17432,
 18947,
 18947,
 29113,
 15290,
 1194,
 16856,
 14150,
 29740,
 29436,
 15290,
 29745,
 29113,
 1208,
 29436,
 15290,
 23925,
 22919,
 16856,
 10325,
 23925,
 14150,
 10325,
 1193,
 19865,
 1181,
 29113,
 29436,
 10260,